In [87]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
from urllib.request import Request, urlopen, urlparse
import re
import time
import requests
import validators

In [88]:
url ='https://www.kickstarter.com/discover/advanced?raised=2&sort=end_date&seed=2743718&page=1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [89]:
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')

In [90]:
def engine(pages) : 
    url = url ='https://www.kickstarter.com/discover/advanced?raised=2&sort=enddate&seed=2743718&page='
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    lst = []
    for i in range(1,pages) :
        r = requests.get(url+str(i),headers=headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        lst.append(soup)
        i =+ 1   
    return lst

In [91]:
containers = soup.find_all('div', class_="js-react-proj-card")

In [92]:
def scraper_prototype(text_profile, prefix):

    if '"name"' in text_profile:
        text_profile = re.sub('"', '', text_profile)

    text2 = text_profile.split(',')
    dummy2 = [re.findall('[A-Za-z0-9_]+:', ww) for ww in text2]
    itemname = [dum_part[0] for dum_part in dummy2 if len(dum_part) > 0]
    ans_array = [re.search(xxy1+'(.*?)'+xxy2, text_profile).group(1).rstrip(',')
                 for xxy1, xxy2 in zip(itemname[:-1], itemname[1:])]
    itemnames = [x.rstrip(':') for x in itemname]

    info_dict = dict()
    for item, ans in zip(itemnames, ans_array):
        info_dict[prefix+'_'+item] = ans
    return info_dict

In [93]:
def to_string(container) : 
    whole_string = str(container).replace('&amp', '').replace('&quot;', '')
    return whole_string

In [94]:
def get_name(whole_string) : 
    if '"name"' in whole_string:
        project_info = '"name"' + re.findall('"name"(.+?)"creator"', whole_string)[0]
    else:
        project_info = 'name' + re.findall('name(.+?)creator', whole_string)[0]
    project_info_dict = scraper_prototype(project_info, 'project')
    df1 = pd.DataFrame.from_dict(project_info_dict, orient='index').transpose()
    return df1

In [95]:
def get_creator(whole_string) : 
    if '"creator"' in whole_string:
        creator_string = re.findall(r'\"creator\"\:\{(.+?)\}', whole_string)[0]
    else:
        creator_string = re.findall(r'creator\:\{(.+?)\}', whole_string)[0]
    creator_info_dict = scraper_prototype(creator_string, 'creator')
    creator_info_dict['creator_slug'] = creator_info_dict.get('creator_slug', 'Null')
    df2 = pd.DataFrame.from_dict(creator_info_dict, orient='index').transpose()
    
    return df2

In [96]:
def get_location(whole_string) : 
    if '"location"' in whole_string:
        location_string = re.findall(r'\"location\"\:\{(.+?)\}', whole_string)[0]
    else:
        location_string = re.findall(r'location\:\{(.+?)\}', whole_string)[0]
    location_info_dict = scraper_prototype(location_string, 'location')
    df3 = pd.DataFrame.from_dict(location_info_dict, orient='index').transpose()
    return df3

In [97]:
def get_profile(whole_string) : 
    if '"profile"' in whole_string:
        profile_string = re.findall(r'\"profile\"\:\{(.+?)\}', whole_string)[0]
    else:
        profile_string = re.findall(r'profile\:\{(.+?)\}', whole_string)[0]
    profile_info_dict = scraper_prototype(profile_string, 'profile')
    df4 = pd.DataFrame.from_dict(profile_info_dict, orient='index').transpose()
    return df4

In [98]:
def make_a_row(df1,df2,df3) : 
    df = pd.concat((df1, df2, df3), axis=1)
    return df

In [99]:
def row_flow(container) : 
    string = to_string(container)
    name = get_name(string)
    creator = get_creator(string)
    location = get_creator(string)
    #profile = get_profile(string)
    row = make_a_row(name,creator,location)
    return row

In [100]:
row = row_flow(containers[0])
row0 = row_flow(containers[1])
row1 = row_flow(containers[2])
row2 = row_flow(containers[3])
row3 = row_flow(containers[4])
row4 = row_flow(containers[5])
row5 = row_flow(containers[6])
row6 = row_flow(containers[7])
row7 = row_flow(containers[8])
row8 = row_flow(containers[9])
row9 = row_flow(containers[10])
row10 = row_flow(containers[11])

In [101]:
df = pd.concat([row,row0,row1,row2,row3,row4,row5,row6,row7,row8,row9,row10],ignore_index=True)

In [102]:
df = df.loc[:,~df.columns.duplicated()]

In [103]:
len(df)

12

In [104]:
for i in range(0, len(df)):
    
    print(df.iloc[i]['creator_id'])

210781665
503461725
524924361
1854969911
857310971
384894682
686715934
397372697
1653002173
2097521505
2094128097
1349936597


In [105]:
site = []
lst = []
url = 'https://www.kickstarter.com/profile/'

In [106]:
for i in range(0, len(df)) : 
    html = requests.get('https://www.kickstarter.com/profile/'+str(df.iloc[i]['creator_id'])+'/about', headers=headers)
    soup = BeautifulSoup(html.content, 'html.parser')
    for a in soup.find_all('a', href=True):
        lst.append(a['href'])
        site.append(df.iloc[i]['creator_id'])

In [107]:
len(lst), lst

(103,
 ['/?ref=nav',
  '/profile/tardigradepress',
  '/profile/tardigradepress',
  '/profile/tardigradepress/about',
  '/profile/tardigradepress',
  '/profile/tardigradepress/created',
  '/profile/tardigradepress/comments',
  'https://tardigradepress.net',
  '/?ref=nav',
  '/profile/plugoarts',
  '/profile/plugoarts',
  '/discover/advanced?backed=-1&ref=profile_location&state=live&woe_id=2352559',
  '/profile/plugoarts/about',
  '/profile/plugoarts',
  '/profile/plugoarts/created',
  '/profile/plugoarts/comments',
  'https://PLUGOarts.com',
  'https://aTigersTale.com',
  '/?ref=nav',
  '/profile/inkwyrmling',
  '/profile/inkwyrmling',
  '/discover/advanced?backed=-1&ref=profile_location&state=live&woe_id=2487610',
  '/profile/inkwyrmling/about',
  '/profile/inkwyrmling',
  '/profile/inkwyrmling/created',
  '/profile/inkwyrmling/comments',
  'https://www.instagram.com/inkwyrmling/',
  '/?ref=nav',
  '/profile/canaryzindell',
  '/profile/canaryzindell',
  '/discover/advanced?backed=-1&re

In [108]:
dodo = []

In [109]:
def is_valid(url):
    parsed = urlparse(url)
    return bool(parsed.netloc) and bool(parsed.scheme)

In [110]:
for el in lst : 
    if is_valid(el) == True : 
        dodo.append(el)
    else : 
        pass

In [116]:
dodo

['https://tardigradepress.net',
 'https://PLUGOarts.com',
 'https://aTigersTale.com',
 'https://www.instagram.com/inkwyrmling/',
 'https://www.theavadavis.com',
 'https://www,thewaltzfilm.com',
 'https://www.studiovosges.com',
 'https://www.duchessofgrantpark.com',
 'https://myfriendtoothy.wixsite.com/website',
 'https://www.drivethrurpg.com/browse/pub/5679/Jeremy-Hart',
 'https://jeremyhartillos.blogspot.com/',
 'https://www.instagram.com/jeremyhartillos/',
 'https://www.patreon.com/jhillos',
 'https://shininglamppress.com',
 'https://stitcherygeekery.wordpress.com/',
 'https://www.etsy.com/shop/stitcherygeekery',
 'https://www.bearapy.org',
 'https://www.instagram.com/gabechengcomics/',
 'https://twitter.com/gabechengcomics',
 'http://www.almpod.com/',
 'https://gabechengcomics.com']

In [84]:
s = []

In [112]:
for el in dodo : 
    try :
        r = requests.get(el)
        t = str(r.text)
        shopify = t.find('shopify')
        s.append(shopify)
    except ConnectionError :
        pass

ConnectionError: HTTPSConnectionPool(host='www,thewaltzfilm.com', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x124d455b0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [117]:
s

[-1, -1, 362, -1, -1, -1, -1, -1, -1, 362, -1, -1, -1, -1, 362, -1, -1, -1, -1]

In [187]:
for i in range(0,len(containers)) : 
    string =  to_string(containers[i])
    name = get_name(string)
    creator = get_creator(string)
    location = get_location(string)
    profile = get_profile(string)
    my_dict[i] = name,creator,location,profile
    df = pd.DataFrame(my_dict)
    final_df = pd.concat([final_df, df],ignore_index=True)
    i =+ 1

In [240]:
for i in range(1, len(containers)) : 
    final_df = row_flow(containers[0])
    #next_row = row_flow(containers[i])
    final_df = pd.concat([final_df, row_flow(containers[i])],ignore_index=True)
    i =+1